<a href="https://colab.research.google.com/github/JKowalsky/Road2MasterDegree/blob/main/ALL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

!cp -r "/gdrive/MyDrive/Colab Notebooks/Master/datasets/prepared/imdb.csv" "/content"
# TODO: Add more datasets

Mounted at /gdrive


In [3]:
import glob

!rm -rf /content/sample_data/

datasets_paths = glob.glob("/content/*")

['/content/imdb.csv']


In [35]:
import pandas
pandas.set_option('display.width', 180)
import sklearn.feature_extraction.text

def tfidf_dataframe_converter(train_dataframe, test_dataframe):
  vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_features=2000, stop_words="english")

  tfidf_matrix = vectorizer.fit_transform(train_dataframe["Content"])
  tfidf_matrix = tfidf_matrix.toarray()
  train_vectors_dataframe = pandas.DataFrame(tfidf_matrix, columns=vectorizer.get_feature_names_out())
  train_vectors_with_class_dataframe = train_vectors_dataframe.assign(Class=train_dataframe["Class"].tolist())

  tfidf_matrix = vectorizer.transform(test_dataframe["Content"])
  tfidf_matrix = tfidf_matrix.toarray()
  test_vectors_dataframe = pandas.DataFrame(tfidf_matrix, columns=vectorizer.get_feature_names_out())
  test_vectors_with_class_dataframe = test_vectors_dataframe.assign(Class=test_dataframe["Class"].tolist())

  return train_vectors_with_class_dataframe, test_vectors_with_class_dataframe


In [36]:
import pandas
pandas.set_option('display.width', 180)

import sklearn.model_selection

folds = 5

for dataset_path in datasets_paths:
  print(f"Dataset: {dataset_path.split('/')[-1]}")
  dataset_dataframe = pandas.read_csv(dataset_path)
  stratified_kfold = sklearn.model_selection.StratifiedKFold(n_splits=folds, shuffle=True)
  for train_subset_indexes, test_subset_indexes in stratified_kfold.split(dataset_dataframe["Content"], dataset_dataframe["Class"]):
    train_subset_dataframe = dataset_dataframe.iloc[train_subset_indexes]
    test_subset_dataframe = dataset_dataframe.iloc[test_subset_indexes]
    converted_train_subset_dataframe, converted_test_subset_dataframe = tfidf_dataframe_converter(train_subset_dataframe, test_subset_dataframe)



Dataset: imdb.csv
